In [1]:
import numpy as np
import theano
import theano.tensor as T
import scipy.sparse as sp
from theano import sparse
import lasagne
import time
import scipy.stats as stats
from collections import OrderedDict
import sys 
sys.setrecursionlimit(50000)


/Applications/anaconda/lib/python3.5/site-packages/theano/tensor/signal/downsample.py:6: UserWarning: downsample module has been moved to the theano.tensor.signal.pool module.
  "downsample module has been moved to the theano.tensor.signal.pool module.")


In [2]:
# Generate a matrix from a multivariate normal distribution with low-rank covariance matrix
KTRUE = 5
K = 20
N = 100
D = 50
maxit = KTRUE*(N+D-KTRUE)
#maxit = N*D

np.random.seed(seed=10)

#c0np = np.random.gamma(1,1)
#gamma0np = np.random.gamma(1,1)
#gammanp = np.random.gamma(1,1)
#rnp = np.random.gamma(gamma0np/KTRUE, c0np, size=(KTRUE))
#w = np.zeros((D,KTRUE))
#for k in range(KTRUE):
#    print(gammanp*rnp[k])
#    print(gammanp)
#    w[:,k] = np.random.gamma(gammanp*rnp[k]+1e-20, gammanp+1e-20, size = (D))
    

w   = np.random.uniform(low=0.0, high=1.0, size=(D,KTRUE))
#Introduce some complexity into w
#w     = np.random.beta(a = 2, b = 5, size = (D,KTRUE))
#maskw = stats.bernoulli.rvs(0.8, size=(D,KTRUE))
#w     = np.multiply(w,maskw)
var = 0.1
covnp = w.dot(w.T)+var*np.eye(D)

Mnp = np.random.multivariate_normal(np.zeros(D), covnp, N).T

In [3]:
maxit

725

In [4]:
random_w_init = np.random.uniform(low=0.01, high=2.0, size=(D,K))
random_r_init = np.random.uniform(low = 0.01, high = 2.0, size = K)

In [5]:
# We use Theano for our model
srng = T.shared_randomstreams.RandomStreams(seed=120)

#Define Theano Variables
Shared = lambda shape,name: theano.shared(value = np.ones(shape,dtype=theano.config.floatX),
                                          name=name,borrow=True) 

In [6]:
# Let Ynp represent our matrix of partial, noisy observations
#p = 0.03
p      = 8*(maxit/(N*D))/(20)
observe = np.random.permutation(np.arange(N*D))[1:int(np.floor(p*N*D))]
Masknp = np.zeros(N*D)
Masknp[observe] = 1
observerow = np.random.randint(0, high=D, size=np.maximum(N,D))
observecol = np.arange(N)
Masknp = Masknp.reshape((D,N))
Masknp[observerow, observecol] = 1
#p = 0.08
#Masknp = np.random.binomial(N, p, size=(N,D)).T
#Masknp = stats.bernoulli.rvs(p, size=(D,N))
#Masknp[1,:] = 1
Mask   = T.as_tensor_variable(Masknp)
M      = T.as_tensor_variable(Mnp)
Y      = Mask*M
zeroY  = T.as_tensor_variable(np.zeros((D,N)))
zero2  = T.as_tensor_variable(np.zeros((D,D)))
zero   = T.as_tensor_variable(np.zeros(D))
st     = T.sum(T.neq(Y, zeroY), axis = 0)
s      = st.eval()


In [7]:
p

0.057999999999999996

In [8]:
#Define variables 
W      = Shared((D,K), 'W')
W.set_value(random_w_init)
r      = Shared((K), 'r')
r.set_value(random_r_init)
Gamma  = Shared((1), 'Gamma')
Gamma0 = Shared((1), 'Gamma0')
c0     = Shared((1), 'c0')
sigma  = Shared((1), 'sigma')

t      = T.dscalar('t')

#Define random variables for MVNscan component
zY = srng.normal([D])
zK = srng.normal([K])

#For data given seqentially we need a different covariance matrix for each yn
WWT=T.dot(W, W.T)
Cov=Shared((D,D), 'Cov')
Cov=WWT+sigma[0]*T.identity_like(WWT)
#Cov = T.as_tensor_variable(covnp, name= 'Cov')

#Define lists
mParams = [W, r, Gamma, Gamma0, c0, sigma]

#indexlist = Shared([maxit], 'indexlist')
indexlist = theano.shared(value = np.zeros([maxit],dtype=np.int64),
                                          name='indexlist',borrow=True)


In [9]:
#Define Functions for Langevin Step

def logJointScanFn(n, logLikelihood, Y, Cov, s):
    
    idxs          = T.neq(Y[:,n], zero).nonzero()
    y             = Y[:,n][idxs]
    idxs2         = T.neq(T.outer(Y[:,n], Y[:,n]), zero2).nonzero()
    littlecov     = Cov[idxs2].reshape((s[n], s[n]))
    logLikelihood +=(-1/2.0)*T.log(T.nlinalg.Det()(littlecov))-(1/2.0)*T.dot(y.T, T.dot(T.nlinalg.MatrixInverse()(littlecov), y))
    
    return logLikelihood

def LogJ(mParams, Y, Cov, s):

    W, r, Gamma, Gamma0, c0, sigma = mParams
    #LogJt0=time.clock()
    results, updates = theano.scan(fn=logJointScanFn,
                                   sequences = np.arange(N),
                                   outputs_info=[dict(initial= np.float64(0) ,taps=[-1])],
                                   non_sequences=[Y, Cov, s])
    logJoint  = results[-1]
    logJoint2 = ((D*Gamma*T.log(Gamma))[0]*r).sum()-(D*T.gammaln(Gamma[0]*r)).sum()+((Gamma[0]*r-1)*T.log(W)).sum()-(Gamma[0]*W).sum() + (Gamma0*T.log(c0)-K*T.gammaln(Gamma0/K)+(Gamma0/K-1)[0]*(T.log(r)).sum()-(c0[0]*r).sum()-Gamma-Gamma0-c0)[0]
    logJoint  += logJoint2

    return(logJoint)

def adadelta2(loss_or_grads, params, learning_rate=1.0, rho=0.95, epsilon=1e-6):
    """ 
    References
    ----------
    .. [1] Zeiler, M. D. (2012):
           ADADELTA: An Adaptive Learning Rate Method.
           arXiv Preprint arXiv:1212.5701.
    """
    grads = get_or_compute_grads(loss_or_grads, params)
    updates = OrderedDict()

    # Using theano constant to prevent upcasting of float32
    one = T.constant(1)

    for param, grad in zip(params, grads):
        value = param.get_value(borrow=True)
        # accu: accumulate gradient magnitudes
        accu = theano.shared(np.zeros(value.shape, dtype=value.dtype),
                             broadcastable=param.broadcastable)
        # delta_accu: accumulate update magnitudes (recursively!)
        delta_accu = theano.shared(np.zeros(value.shape, dtype=value.dtype),
                                   broadcastable=param.broadcastable)

        # update accu (as in rmsprop)
        accu_new = rho * accu + (one - rho) * grad ** 2
        updates[accu] = accu_new

        # compute parameter update, using the 'old' delta_accu
        update = (grad * T.sqrt(delta_accu + epsilon) /
                  T.sqrt(accu_new + epsilon))
        #updates[param] = param - learning_rate * update
        updates[param] = T.minimum(T.maximum((param - learning_rate * update).astype(theano.config.floatX), (1e-10)*T.ones_like(param)), 10*T.ones_like(param))

        # update delta_accu (as accu, but accumulating updates)
        delta_accu_new = rho * delta_accu + (one - rho) * update ** 2
        updates[delta_accu] = delta_accu_new

    return updates

def get_or_compute_grads(loss_or_grads, params):
    """Helper function returning a list of gradients
    """
    if any(not isinstance(p, theano.compile.SharedVariable) for p in params):
        raise ValueError("params must contain shared variables only. If it "
                         "contains arbitrary parameter expressions, then "
                         "lasagne.utils.collect_shared_vars() may help you.")
    if isinstance(loss_or_grads, list):
        if not len(loss_or_grads) == len(params):
            raise ValueError("Got %d gradient expressions for %d parameters" %
                             (len(loss_or_grads), len(params)))
        return loss_or_grads
    else:
        return theano.grad(loss_or_grads, params)
    
    
#MVNormalScan constructs our estimate of the entire matrix using conditional multivariate normal

def MVNormalScan_beta02(n, Y, Mask, Cov, W, zY, zK, s):
    
    #construct binaryY_unobs a vector of 1s and 0s where the ith coord is a 1 if we haven't seen the ith coord of y_n    
    binaryY_unobs = T.eq(Y[:,n], zero)
    #construct covariance of the observed entries where the rows/columns with nothing have a 1 on diag (so invertible)
    idxs          = T.neq(Y[:,n], zero).nonzero()
    y             = Y[:,n][idxs]
    idxs2         = T.neq(T.outer(Y[:,n], Y[:,n]), zero2).nonzero()
    littlecov     = Cov[idxs2].reshape((s[n], s[n]))
    littlecov_inv = T.nlinalg.MatrixInverse()(littlecov)
    
    #sigma_observed     = T.outer(binaryY[:,n], binaryY[:,n])*Cov+(binaryY_unobs*T.identity_like(Cov))
    sigma_unobs_obs         = (T.outer(binaryY_unobs, T.neq(Y[:,n], zero)))*Cov
    idxs3                   = T.neq(sigma_unobs_obs, zero2).nonzero()
    little_sigma_unobs_obs = sigma_unobs_obs[:,idxs].reshape((D, s[n])) 
    #sigma_observed_inv = T.nlinalg.MatrixInverse()(sigma_observed)
    dummyY             = T.zeros(D)
    
    #draw the mean vector dummyY from N(0, WWT+sigma^2I) using computationally fast trick
    dummy_results, dummy_updates= theano.scan(lambda prior_result, sigma, zY, W, zK: 
                                              T.sqrt(sigma)[0]*zY+T.dot(W,zK) + prior_result,
                                              sequences=None,
                                              outputs_info= T.zeros(D),
                                              non_sequences=[sigma, zY, W, zK],
                                              n_steps=R)
    
    dummyY       = dummy_results[-1]
    dummyY       /= R
    dummyY_obs   = dummyY[idxs]
    dummyY_unobs = binaryY_unobs*dummyY
    y_est        = dummyY_unobs + T.dot(T.dot(little_sigma_unobs_obs, littlecov_inv), (y-dummyY_obs))
    y_est        = (y_est*binaryY_unobs)-(1e6)*Mask[:,n]
    #y_est        = y_est*binaryY_unobs + Y[:,n]*Mask[:,n]
    
    return [y_est, sigma_unobs_obs, littlecov_inv]


In [10]:
Ytrue = Y.eval()

In [11]:
#Stochastic Gradient Descent

counter = 1

logJ = LogJ(mParams,Y,Cov, s)
ParamUpdates2=adadelta2(-logJ,mParams)
AdaDeltaStep2=theano.function(inputs=[], updates=ParamUpdates2)


In [12]:
#This serves to test how well, given whatever percent of data observed you set through variable "p", 
#we can estimate the covariance matrix
% pylab inline
import numpy as np
import matplotlib.pyplot as plt

limit = 1000

outs, updts = theano.scan(AdaDeltaStep2,
                              n_steps = limit)

fn = theano.function(
[],
outs,updates = updts
)
fn()

#plt.plot(ljlist)

print(r.eval())

"""
nameW = 'WK' + str(KTRUE) +'.csv'
namer = 'rK' + str(KTRUE) + '.csv'
namecov = 'truecovsdK' + str(KTRUE) + '.csv'
np.savetxt(nameW,  W.eval(), delimiter = ',')
np.savetxt(namer,  r.eval(), delimiter = ',')
np.savetxt(namecov,  w.dot(w.T), delimiter = ',')
"""

Populating the interactive namespace from numpy and matplotlib
[ 0.97390015  0.52469324  0.49581771  1.4033288   0.85894179  0.74639297
  1.69236316  0.65655201  1.68411802  0.91000296  0.90574836  1.67776819
  1.16857063  1.21502785  0.42786266  1.83679483  1.03846537  0.27255364
  0.67783298  0.04778285]


`%matplotlib` prevents importing * from pylab and numpy


"\nnameW = 'WK' + str(KTRUE) +'.csv'\nnamer = 'rK' + str(KTRUE) + '.csv'\nnamecov = 'truecovsdK' + str(KTRUE) + '.csv'\nnp.savetxt(nameW,  W.eval(), delimiter = ',')\nnp.savetxt(namer,  r.eval(), delimiter = ',')\nnp.savetxt(namecov,  w.dot(w.T), delimiter = ',')\n"

In [13]:
print(r.eval())

[ 0.97390015  0.52469324  0.49581771  1.4033288   0.85894179  0.74639297
  1.69236316  0.65655201  1.68411802  0.91000296  0.90574836  1.67776819
  1.16857063  1.21502785  0.42786266  1.83679483  1.03846537  0.27255364
  0.67783298  0.04778285]


In [14]:
def MainBandit2(count, indexlistnp, Mask, W, r, sigma, Gamma, Gamma0, c0, Y):
    
    limit = 1000
    R     = 10
    construct_lj_list = 0
    construct_error_list = 0
    tt = time.clock()
    outs, updts = theano.scan(AdaDeltaStep2,
                              n_steps = limit)

    fn = theano.function(
    [],
    outs,updates = updts
    )
    fn()
    
    #[ljlist, errlist, W, r, sigma, Gamma, Gamma0, c0] = SGD(Y, limit,construct_lj_list, construct_error_list, W, r, sigma, Gamma, Gamma0, c0)
    
    tt = time.clock()
    [y_estimate, sigma_u_o_scan, sigma_ob_inv_scan], updates=theano.scan(fn=MVNormalScan_beta02,
                                              sequences=T.arange(N),
                                              outputs_info=None,
                                              non_sequences=[Y, Mask, Cov, W, zY, zK, st])
    
    
    tt=time.clock()
    y_estimate     = y_estimate.T
    #print('y_estimate')
    #print(y_estimate[1,:].eval())
    [value, index] = T.max_and_argmax(y_estimate, axis=None, keepdims=False)   
    mf             = T.flatten(Mask)
    mf             = T.inc_subtensor(mf[index],1)
    Mask           = mf.reshape((D,N))
    #print('index')
    #print(index.eval())
    
    #ON CPU IT MIGHT BE FASTER TO EVAL INDEX EACH ITERATION, BUT PERHAPS USE THEANO INDEXLIST FOR GPU 
    
    #indexlist      = T.set_subtensor(indexlist[count], index)
    indexlistnp.append(index.eval())
    #print(r.eval())
    return indexlistnp,Mask, W, r, sigma, Gamma, Gamma0, c0

In [ ]:
def ind2sub(array_shape, ind):
    #ind[ind < 0] = -1
    #ind[ind >= array_shape[0]*array_shape[1]] = -1
    rows = np.floor(ind / array_shape[1])
    cols = ind % array_shape[1]
    return (int(rows), int(cols))

In [ ]:
count     = 0
ratings   = [] 
R         = 5

"""
[indices, Mask_evolve, W_evolve, r_evolve, sigma_evolve, Gamma_evolve, Gamma0_evolve, c0_evolve], updates = theano.scan(fn=MainBandit2,
                                               sequences = T.arange(5),
                                               outputs_info = [indexlist,Mask, W, r, sigma, Gamma, Gamma0, c0],
                                               non_sequences = Y)
                                               
indexlistnp = indices[-1].eval()
"""
indexlistnp = []
for i in range(maxit):
    [indexlistnp, Mask, W, r, sigma, Gamma, Gamma0, c0]  = MainBandit2(count, indexlistnp, Mask, W, r, sigma, Gamma, Gamma0, c0, Y)
    count = count + 1
    if i%10 == 0:
        print(i)
    #print(r.eval())


0
10
20
30
40


In [ ]:

#indexlist[-1].eval()

In [ ]:
#indexlistnp = indexlist[1:1000].eval()
#indexlist[-1].eval()
ratings = []
initial_obs = np.multiply([Masknp!=0], Mnp).flatten()
ratings = initial_obs[initial_obs!=0].tolist()
for i in range(np.size(indexlistnp)):
    [r,c] = ind2sub(np.shape(Mnp), int(indexlistnp[i]))
    ratings.append(Mnp[r,c])

In [ ]:
indexlistnp[0:40]

In [ ]:
reward = np.cumsum(ratings)

In [ ]:
#best = Mnp + 0.00001
#best[Masknp] = 1e-6
#best = best.flatten()
#(np.multiply(Mnp, (1e-6)*Masknp)).flatten()
best = Mnp.flatten()
best.sort()
best[:] =  best[::-1]
best = np.cumsum(best)

random_reward = np.zeros(np.size(Mnp.flatten()))
for i in range(10):
    #random = Mnp + 0.00001
    #random[Masknp] = 1e-6
    #random =  random.flatten()
    #random = (np.multiply(Mnp, (1e-6)*Masknp)).flatten()
    random = Mnp.flatten()
    random = np.random.permutation(random)
    random_reward += np.cumsum(random)/10

In [ ]:
#PLOT ACCUMULATED REWARDS

% pylab inline
import numpy as np
import matplotlib.pyplot as plt

length        = np.minimum(np.size(reward), np.size(best))
reward        = reward[0:length]
best          = best[0:length]
random_reward = random_reward[0:length]

plt.plot(best)
plt.plot(reward)
plt.plot(random_reward)

In [ ]:
indexlistnp

In [ ]:
#PLOT REGRET

plt.plot(best-reward)
plt.plot((best-random_reward))

In [ ]:
r

In [ ]:
maxit

In [ ]:
N*D